In [2]:
import numpy as np

In [ ]:
# for wap
def calc_wapArea(wap500, regime):
    if regime == 'area_ascent':
        areaFrac = ((wap500<0)*1).sum(dim=('lat','lon'))/(wap500.shape[1]*wap500.shape[2])*100
        
    if regime == 'area_descent':
        areaFrac = ((wap500>0)*1).sum(dim=('lat','lon'))/(wap500.shape[1]*wap500.shape[2])*100
        
    areaFrac.attrs['units'] = '%'
    return areaFrac

In [ ]:
# assign variables to a dataset
dlats = xr.Dataset(
    )

dlons = xr.Dataset(
    )

res = xr.Dataset(
    )

for model in models_cmip6:
        data = get_dsvariable('precip', model, experiments[0], resolution=resolutions[0])['precip']
        
        dlats[model] = data['lat'][2] - data['lat'][1]
        dlons[model] = data['lon'][2] - data['lon'][1]
        res = dlats[model] * dlons[model]

In [ ]:
# another way to plot a series of colors
import numpy as np
import matplotlib.pyplot as plt
import itertools

m = 5
n = 5

x = np.zeros(shape=(m, n))
plt.figure(figsize=(5.15, 5.15))
plt.clf()
plt.subplot(111)
marker = itertools.cycle(('o', 'v', '^', '<', '>', 's', '8', 'p'))
ax = plt.gca()
for i in range(1, n):
    x = np.dot(i, [1, 1.1, 1.2, 1.3])
    y = x ** 2
    color = next(ax._get_lines.color_cycle)
    plt.plot(x, y, linestyle='', markeredgecolor='none', marker=marker.next(), color=color, label = str(i))
    plt.plot(x, y, linestyle='-', color = color)
plt.axvline(2.)
plt.axvline(3.)
plt.axhline(4.)
plt.axhline(6.)
plt.ylabel(r'y', labelpad=6)
plt.xlabel(r'x', labelpad=6)
plt.show()

In [ ]:
## colormap new for plotting
from matplotlib.cm import get_cmap
name = "Accent"
cmap = get_cmap('tab10')
colors = cmap.colors
ax.set_prop_cycle(color=colors)

# plt.rcParams['figure.figsize'] = [8, 8]
# plt.rcParams.update({'font.size': 18})


In [ ]:
#create binary arrays
binary_matrix = np.array([
    [0, 1, 0],
    [1, 0, 1],
    [0, 1, 0]
])

In [ ]:
# aligning variables if they have one common coordinate, to be able to create a dataset with including multiple variables with differing dimensions

import xarray as xr

# Assume 'temp' has dimensions ('time', 'lev', 'lat', 'lon')
# and 'precip' has dimensions ('time', 'lat', 'lon')
temp_aligned, precip_aligned = xr.align(temp, precip, join='outer')

# Create xarray dataset with two variables
ds = xr.Dataset(
    {
        'temperature': temp_aligned,
        'precipitation': precip_aligned
    },
    coords={
        'time': temp.time,
        'lev': temp.lev,
        'lat': temp.lat,
        'lon': temp.lon
    }
)

In [ ]:
# # Detrend the data, remove linear or polynomial trend
# from scipy.signal import detrend
# anomalies_detrended = xr.apply_ufunc(
#     detrend, anomalies_deseasonalized,
#     kwargs={'axis': 0}, input_core_dims=[['time']], output_core_dims=[['time']]
# )

# title = 'detrended anomaly from {}, model:{}, experiment:{}'.format('vertical mean hur', datasets[0], experiments[0])
# xlabel = '{} anomaly [{} - {}]'.format(timeMean_options[0], str(hur.isel(time=6).coords['time'].values)[:10], str(hur.isel(time=-6).coords['time'].values)[:10])
# ylabel = '{} [{}]'.format('Relative humidity', hur.units)

# plot_timeseries(anomalies_detrended, title=title)
# plt.xlabel(xlabel)
# plt.ylabel(ylabel)

# # Define the degree of the polynomial for detrending
# poly_degree = 2

# # Fit a polynomial function to the time series
# time = np.arange(len(anomalies_deseasonalized['time']))
# data = anomalies_deseasonalized.values
# p = np.polyfit(time, data, poly_degree)

# # Subtract the polynomial function from the original data
# trend = np.polyval(p, time)
# anomalies_detrended = anomalies_deseasonalized - trend

# # Assign time coordinate to detrended data array
# anomalies_detrended = anomalies_detrended.assign_coords(time=anomalies_deseasonalized['time'])

# plot_timeseries(anomalies_detrended, title=title)
# plt.xlabel(xlabel)
# plt.ylabel(ylabel)

In [ ]:
# moving subplots
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0-0.0175, pos.width, pos.height])

In [ ]:
# cell with example of one model, and obs
# create a function to resample the data. Then apply function to the desired variable
timeMean_options = [
    # 'annual'
    # 'month'
    # 'season'
    # 'season_mean'
    'month_mean'
    # 'day'
    ]
timeMean_option = timeMean_options[0]

var = tas_sMean
variable_name = variables[1]

plot_model= True
if plot_model:
    plt.figure(figsize=(25,5))
    
    title = variable_name + ', '+ timeMean_option + ', ' + model + ', ' + experiment

    if timeMean_option == 'year':
        y = var.resample(time='Y').mean(dim='time', keep_attrs=True)

        plot_timeseries(y, variable_name, timeMean_option, title)

    if timeMean_option == 'month':
        y = var.resample(time='M').mean(dim='time', keep_attrs=True)

        plot_timeseries(y, variable_name, timeMean_option, title)

    if timeMean_option == 'day':
        y = var

        plot_timeseries(y, variable_name, timeMean_option, title)
    
    if timeMean_option == 'season':
        y = var.resample(time='QS-DEC').mean(dim="time")
        y = to_monthly(y)
        y = y.rename({'month':'season'})
        y = y.assign_coords(season = ["MAM", "JJA", "SON", "DJF"])

        y.plot(x='year', hue='season', figsize=(25,5))
        plt.title(title)

    if timeMean_option == 'season_mean':
        y = var.resample(time='QS-DEC').mean(dim="time")
        y = to_monthly(y)
        y = y.rename({'month':'season'})
        y = y.assign_coords(season = ["MAM", "JJA", "SON", "DJF"])

        plt.figure(figsize=(10,2.5))
        ax = plt.gca()
        (y.mean(dim='year') - y.mean(dim='year').mean(dim='season')).to_series().plot.bar(ax=ax)
        plt.axhline(y=0, color='k', linestyle='--')

        ax.set_xticklabels(y.season.values, rotation=30, ha='right')
        plt.title(title  + ', anomaly from annual mean')


    if timeMean_option == 'month_mean':
        y = to_monthly(var)
        y = y.assign_coords(month = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul','aug', 'sep', 'oct', 'nov', 'dec'])
        y.mean(dim='year').plot(figsize = (10,2.5))
        plt.title(title)



plot_obs = False
if plot_obs:
    var_obs = ''
    title = variable_name + ', '+ timeMean_option + ', ' + obs + ', ' + experiment

    if timeMean_option == 'year':
        y = var_obs.resample(time='Y').mean(dim='time', keep_attrs=True)

        plot_timeseries(y)

    if timeMean_option == 'month':
        y = var_obs.resample(time='M').mean(dim='time', keep_attrs=True)

        plot_timeseries(y, variable_name, timeMean_option, title)

    if timeMean_option == 'day':
        y = var_obs

        plot_timeseries(y, variable_name, timeMean_option, title)
    
    if timeMean_option == 'season':
        y = var_obs.resample(time='QS-DEC').mean(dim="time")
        y = to_monthly(y)
        y = y.rename({'month':'season'})
        y = y.assign_coords(season = ["MAM", "JJA", "SON", "DJF"])

        y.plot(x='year', hue='season', figsize=(25,5))
        plt.title(title)

    if timeMean_option == 'season_mean':
        y = var_obs.resample(time='QS-DEC').mean(dim="time")
        y = to_monthly(y)
        y = y.rename({'month':'season'})
        y = y.assign_coords(season = ["MAM", "JJA", "SON", "DJF"])

        plt.figure(figsize=(10,2.5))
        ax = plt.gca()
        (y.mean(dim='year') - y.mean(dim='year').mean(dim='season')).to_series().plot.bar(ax=ax)
        plt.axhline(y=0, color='k',linestyle='--')

        plt.title(obs + ', anomaly from annual mean')
        ax.set_xticklabels(y.season.values, rotation=30, ha='right')


    if timeMean_option == 'month_mean':
        y = to_monthly(var_obs)
        y = y.assign_coords(month = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul','aug', 'sep', 'oct', 'nov', 'dec'])
        y.mean(dim='year').plot(figsize = (10,2.5))
        plt.title(title)


In [ ]:
# x- and y-labels
+ ' ['+y.units+']'

In [ ]:
# set figsize externally
fig.set_size_inches(figsize[0], figsize[1])

In [ ]:
# variable number of rows based on nnumber of datasets to plot
fig= plt.figure(figsize=(22,12))
n_models = len(models)
n_cols = 4
n_rows = (n_models + n_cols - 1) // n_cols
ax= fig.add_subplot(n_rows, n_cols, i + 1, projection=cartopy.crs.PlateCarree(central_longitude=180))

# how to change figure size corresponding to how many rows are included?

In [ ]:
# print date in title
plt.title("Time = " + str(tas.coords['time'].values[i])[:13])


In [ ]:
# timing
print(model, 'started') 
start = timeit.default_timer()
stop = timeit.default_timer()
print('model: {} took {} minutes to finsih'.format(model, (stop-start)/60))

In [ ]:
# rename dimensions
precip = precip.rename({'latitude': 'lat', 'longitude': 'lon'})

In [22]:
# copy array in new dimension
a = np.ones(shape = (3,4))
b = np.repeat(a[np.newaxis,...], 3, axis=0)

In [ ]:
# broadcasting
a = np.ones(shape=(2,3,4))
b = np.array([1,2])
c = a*b[...,None, None]
c = a*b[:,np.newaxis, np.newaxis]
a*np.expand_dims(b,axis=(1,2))

In [ ]:
# define xarray data array and data set
pr999 = precip.quantile(0.999,dim=('lat','lon'),keep_attrs=True)
pr999 = xr.DataArray(
    data = pr999.data,
    dims = ['time'],
    coords = {'time': precip.time.data},
    attrs = {'units':'mm/day'}
    )
    

pr_percentiles = xr.Dataset(
    data_vars = {'pr95': pr95, 
                    'pr97': pr97, 
                    'pr99': pr99, 
                    'pr999': pr999}
    ) 

In [ ]:
# creating a seasonal plot
season = pr_percentile.resample(time='QS-DEC').mean(dim="time")
season = to_monthly(season)
# 3 - MAM
# 6 - JJA
# 9 - SON
# 12 - DJF
season = season.rename({'month':'season'})
season = season.assign_coords(season = ["MAM", "JJA", "SON", "DJF"])
season.plot(x='year', hue='season', figsize=(25,5))
plt.title(percentile_option + ', model: ' + model + ', ' + experiment)

In [1]:
# writing superscripts in string
print('precipitation [mm day' + chr(0x207B) + chr(0x00B9) + 'K'  + ' \u00B0C ' + chr(0x207B) + chr(0x00B9) + ' km\u00b2]')

precipitation [mm day⁻¹K °C ⁻¹km²]


In [ ]:
# groupby
pr_percentileSeason = pr_percentile.groupby('time.season').mean(dim='time')
pr_percentileMonth = pr_percentiles.pr99.resample(time='M').mean(dim='time')
mse_monthly = pr_percentile.groupby('time.month').mean(dim ='time').plot()

In [ ]:
# animation
# from matplotlib import pyplot as plt, animation
# from IPython.display import HTML, display

# animate live in jupyter cell
# HTML(ani.to_jshtml())

# display saved mp4 in jupyter notebook with:
# display(HTML("<video controls><source src='/Users/cbla0002/Documents/data/cmip5/animations/GFDL-CM3_location_high_pr_percentile_historical.mp4' type='video/mp4'></video>"))

# save mp4 using ffmpeg
# folder = home + '/Documents/data/cmip5/animations'
# fileName = model + '_location_high_pr_percentile_' + experiment + '.mp4'
# path = folder + '/' + fileName
# ani.save(path)

In [ ]:
# dedault blue color: '#1f77b4'